In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from datetime import datetime,timedelta
import sys
# sys.path.insert(0, '../src/')

from labeling_utils import stem_set,read_file_properties,find_files,ffmpeg_split_mp3
import pandas as pd
from pydub import AudioSegment
from pathlib import Path

In [ ]:
/tank/data/nna/real/
prudhoe_23_2019_101RECNX_RCNX0324.jpg

region/locationId/year/recorderId_startDateTime.ext

In [ ]:
S4A10292_20190507_233000.flac

In [6]:
from PIL import Image
from PIL.ExifTags import TAGS

def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val

    return labeled

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

exif = get_exif('/tank/data/nna/examples/randomPhotos10k/anwr_33_2019_101RECNX_RCNX3238.JPG')
labeled = get_labeled_exif(exif)
# print(labeled)


In [11]:
# labeled[""]
# labeled.keys()
labeled["DateTimeDigitized"]


('2019:06:17 16:25:32', '2019:06:17 16:25:32')

In [ ]:
def display_audio(sorted_filtered):
    from IPython.display import display
    for f in sorted_filtered.iterrows():
        print(f[0],)
    #     print("start",start_t,)
    #     print("end",end_t)
    #     print("timestamp",f[-1].timestamp)
    #     print((start_t-f[-1].timestamp))

        start_seconds= max(((start_t-f[-1].timestamp)).total_seconds(),0)
        end_seconds = max((end_t-f[-1].timestamp).total_seconds(),0)

#         print("test",start_seconds,end_seconds)

        mp3_file_path=f[0]
        tmpfolder="./tmp/"
        # if end_seconds bigger than file, ffmpeg ignores it, if both out of order than output is emtpy
        ffmpeg_split_mp3(mp3_file_path,start_seconds,end_seconds,tmpfolder=tmpfolder)
    #     if start_seconds>0:
    #         files.append(mp3_split(f[0],start_second=start_seconds,end_second=start_seconds+end_seconds))
        try:
            mp3file=AudioSegment.from_mp3(Path(tmpfolder+"output.MP3"))
            display(mp3file)
        except:
            print("{}".format(sys.exc_info()[0]))
        print("--------------")


In [ ]:
# TODO use a file that is permanent, or create one on the fly

mp3files_path="/home/enis/projects/nna/src/mp3files.txt"
mp3files,ignored=stem_set(mp3files_path)
file_properties,exceptions=read_file_properties(mp3files)
file_properties_df=pd.DataFrame(file_properties).T

In [ ]:
# site_id or site name (some files are missing site_names)
location="CLVL4"
# DD-MM-YYYY_HH:MM:SS
start_time='22-08-2016_20:33:00' # or datetime object
length=1000 # seconds of audio, if set end_time is IGNORED
end_time=''  # or datetime object

sorted_filtered,start_t,end_t=find_files(location,start_time,end_time,length,file_properties_df)
display_audio(sorted_filtered)